## **(1) Big picture of each script** ##

- **My script:**

> Makes a full-screen white window, loads T and L images, shows a fixation, and presents instructions with explicit keys 1 = present and 0 = absent.

> Then practice (untimed with feedback), then main trials (timed with a Clock), and saves results at the end of the experiment.

> The coordinate maps are pre-centered for set sizes 3/6/9, which makes layouts predictable and visually clean.

- **Classmates script:**

> Starts with a GUI dialog to grab SubjectID and trials per condition and immediately opens a CSV with a header (so every run is labelled).

> Uses functions for placing stimuli (pos_and_ori), deciding present/absent (targ_pres), and response collection (KeyGet).

> Then runs practice + main blocks and shows a final summary screen (average RT, accuracy, and number of misses).

## **(2) What I liked/learned from my classmates script + some things to change** ##

#### **(A) Quick startup using GUI + immediately opening a labeled CSV** ####

In [ ]:
dlg = gui.Dlg()
dlg.addField('SubjectID:')
dlg.addField('Trials Per Cond:')
ok_data = dlg.show()
if not dlg.OK:
    core.quit()

sub_ID = ok_data[0]
trials = int(dlg.data[1])
fileName = sub_ID + "_" + expName
dataFile = open(fileName + '.csv', 'w')
dataFile.write('SetSize,TP, RT, Correct, Missed\n')

- **I like this approach because it makes the script interactive and flexible.**

> Instead of hardcoding the participant IDs or number of trials, this uses gui.Dlg() to collect those values when the experiment starts.

> It’s a small change that makes the code easier to reuse for different participants without editing the script every time.

- **It also sets up the data file right away, with a header line.**

> That’s really smart for data management, because it means each participant automatically gets their own CSV labelled with their Subject ID (from fileName = sub_ID + "_" + expName).

> If the script crashes or is stopped mid-run, you would still have at least a partial dataset with proper column names, which is really helpful during debugging or pilot testing.

> This is something that I did not do in my own script, but I can see the importance now (i.e., if the experiment crashes/stopped, I would lose all my data).

- **The structure makes the “save pathway” visible early in the code.**

> I can tell exactly where data is being saved and under what name, which helps prevent overwriting previous runs.

> I like that it’s formatted as 'SetSize,TP, RT, Correct, Missed\n'—those headers directly match the variables that get saved later, which will make imports to things like SPSS or Excel easy.

- **Tiny improvement (what I learned to consider):**

> **I might wrap the file creation line in a context manager:**

> with open(fileName + '.csv', 'w') as dataFile:
          dataFile.write('SetSize,TP, RT, Correct, Missed\n')

> That way, the file will automatically close even if PsychoPy crashes or the experimenter ends early.

> It’s a small thing, but it’s considered best practice for file safety.

- **Broader takeaway I learned:**

> This snippet of code reminded me that collecting participant info and setting up data output at the top of a PsychoPy script helps organize the whole experiment flow.

> It keeps participant-specific setup separate from the trial logic, and that makes debugging easier.

#### **(B) Randomized positions + orientations helper (more compact stimulus drawing)** ####

In [ ]:
def pos_and_ori(target, distract, samp_size):
    samplelist = list(range(-180, 180, 25))
    x = random.sample(samplelist, samp_size)
    y = random.sample(samplelist, samp_size)
    for n in range(0, samp_size - 1):
        orientations = [0, 90, 180, 270]
        orin = random.choice(orientations)
        distract.ori = orin
        distract.pos = (x[n], y[n])
        distract.draw()
    for n in range(samp_size - 1, samp_size):
        target.pos = (x[n], y[n])
        target.draw()
    return distract, target

- **What this function does:**

> This function neatly bundles all the “draw stimuli on the screen” steps into one reusable piece of code.

> It randomly selects a set of (x, y) coordinates, assigns each distractor (the Ls) a random orientation, and then draws exactly one target (the T) in the final coordinate slot.

> By wrapping this in a function, the main loop becomes much shorter — I just call pos_and_ori() instead of repeating the same drawing logic for every trial.

> It’s a really efficient way to keep the main experiment flow clean.

- **What I liked/learned:**

> I like how this uses two loops so that the last item automatically becomes the target.

> That’s a simple but smart way to separate distractors and the target without needing a “target index” variable like I included in my own script.

> I also learned that using random.sample() avoids duplicate positions, reducing the chance that shapes overlap or cluster.

- **Two things to keep in mind (both myself and my classmate):**

> **(1)** This code assumes exactly one target per trial and fills the final slot with that target. That’s perfect for a classic visual search task like this, but if I wanted to manipulate target frequency or have multiple targets, you would need to change the logic that decides how many distractors vs. targets are drawn.

> **(2)** Because the x and y coordinates are sampled independently, it’s possible that some positions line up too neatly (e.g., a grid-like pattern). If you wanted a more controlled spacing (equal distance between items), I might generate coordinates from a grid and randomly shuffle them instead of sampling x and y separately.

#### **(C) Clear, self-contained “present vs absent” decision (single entry point)** ####

In [ ]:
def targ_pres(trial_list, total_trials, distract, target, condition_index):
    pres_or_not = random.choice(trial_list)
    trial_list.remove(pres_or_not)
    if pres_or_not <= np.median(total_trials):
        targ_there = 0
        stimuli = pos_and_ori(distract, distract, condition)
    else:
        targ_there = 1
        stimuli = pos_and_ori(target, distract, condition)
    return targ_there, stimuli

- **As a beginner, I appreciate a single function that decides present/absent and immediately draws the correct array.**

> I really like that this function handles both deciding whether the target is present and drawing the correct array right away.

> It keeps all of the logic for presence/absence in one place, which reduces the number of moving parts in the main loop.

> I didn’t do this in my own code because I wasn’t sure how to combine those steps, but seeing it here helped me understand how efficient and organized this structure can be.

- **I also learned a few interesting tricks:**

> The use of the median to split the list into half “present” and half “absent” trials is clever and compact. It automatically balances the number of each trial type across a block without needing two separate lists.

>Another thing I learned is how they call pos_and_ori(distract, distract) for absent trials and pos_and_ori(target, distract) for present trials. It’s a short, elegant way to make sure a target is only drawn when it should be.

- **Something to consider changing:**

> Even though this method works well, it’s a bit hard to read at first glance (i.e., I was quite confused for a few minutes).

> From my understanding, it relies on knowing how the trial list and median are used, which might confuse newer coders (aka me).

> A more explicit flag for “present = True/False” could make this even clearer.

> **What I may do:** Possible build a list of presence flags (e.g., [0,1,0,1,...]), shuffle it, and in the main loop do present = flags[i]; then call a draw helper with present=True/False. It’s a tiny bit longer, but to me easier to understand.

#### **(D) Response collection with a per-trial timeout (beginner-friendly loop)** ####

In [ ]:
def KeyGet(trial_duration=2.0, rt=None, resp=None):
    startTime = core.getTime()
    while core.getTime() - startTime < trial_duration and resp is None:
        keys = event.getKeys(keyList=['a', 'd', 'escape'])
        if keys:
            key = keys[0]
            rt = core.getTime() - startTime
            if key == 'a':
                resp = 'a'
                break
            elif key == 'd':
                resp = 'd'
                break
            elif key == 'escape':
                core.quit()
        core.wait(0.01)
    if resp is None:
        resp = 'no_response'
        rt = 999
    return resp, rt

- **What this code does:**

> This loop continually checks for keypresses until time runs out or a response is made.

> It only listens for 'a', 'd', or 'escape', which keeps the input clean and task-specific.

> The trial_duration variable determines how long participants are given to make a response, and the brief 10-ms wait helps reduce CPU load by stopping PsychoPy from running the loop at full speed.

- **What I liked/learned:**

> I like how easy this is to read — it mirrors exactly how I’d describe the logic out loud (i.e., “keep checking until time runs out or I get a response”). The built-in timeout makes it easy to adjust task difficulty by changing one line, and using a small sleep interval keeps the program running smoothly.

> I also really like how this code still saves something even when the participant doesn’t respond. Setting 'no_response' and 999 makes it clear that the trial ended because of a timeout, not because of an error in the code. It’s a simple way to keep the data complete and consistent.

- **Possible improvement:**

> If I were to simplify this later, I might try the event.waitKeys(timeStamped=clock, maxWait=...) method, since it does the same thing in fewer lines (https://www.psychopy.org/api/event.html#psychopy.event.waitKeys).

>  But for learning purposes, this version helped me really understand how the response loop and timing work behind the scenes.

#### **(E) Centralized scoring + feedback message (keeps logic tidy)** ####

In [ ]:
def Response(resp, rt, targ_there):
    if resp == 'd' and targ_there == 1:
        corr = 1
        feedback = 'Correct!'
        response_time = round(rt, 2)
    elif resp == 'a' and targ_there == 0:
        corr = 1
        feedback = 'Correct!'
        response_time = round(rt, 2)
    elif resp == 'd' and targ_there == 0:
        corr = 0
        feedback = 'Incorrect'
        response_time = round(rt, 2)
    elif resp == 'a' and targ_there == 1:
        corr = 0
        feedback = 'Incorrect'
        response_time = round(rt, 2)
    elif resp == 'no_response':
        corr = 0
        feedback = 'No Response'
        response_time = 'NA'
    return corr, feedback, response_time

- **What this code does:**

> This function checks whether the participant’s keypress matches the correct condition and returns three things: whether they were correct, the feedback message, and their reaction time.

> Having all of this in one place means I don’t have to repeat these checks throughout the script.

- **What I liked/learned:**

> **I like that the “is it correct?” logic is all grouped inside one dedicated function.** This is a good programming practice because it keeps the main loop focused on timing and flow while moving decision-making rules into a single, easily readable location.

> **It also makes debugging much easier.** If feedback looks wrong during testing, I only need to inspect one place in the script rather than hunting through multiple blocks. I can change the response rules once and know the change applies everywhere.

>  **Returning all three pieces of information together (i.e., correctness, feedback, and response time) is a tidy design choice.** It keeps my main experiment loop short, and it mirrors how real experiment frameworks (like jsPsych or PsychoPy Builder components) often structure trial outputs.

> **I also learned how helpful it is to work with clear, explicit condition checks when you’re still developing the logic.** Even though this function could likely be shortened, the current version is very readable, almost self-documenting.

- **What I would want to keep in mind:**

> **(1)** If I ever changed which keys represent “present” and “absent,” I could easily update those lines here without needing to edit multiple sections of the script. This flexibility is helpful if the experiment is ported to a different keyboard layout or run online, where key availability likely differs.

> **(2)** In a future version, I might simplify some of the repetition using a dictionary or mapping structure (e.g., mapping target conditions directly to the correct key), which would reduce the number of if/elif blocks. But for now, the explicit version is easy to follow and beginner-friendly.

> **(3)** I should also think about how I’m handling missed responses. Right now, “NA” is returned as a string. Depending on my analysis plans (especially if I’m using SPSS or R later), it might be cleaner to use something like None or a numeric placeholder. But this is more of an analysis preference than a requirement.

> **(4)** Overall, I would just want to remember how helpful it is to isolate logic like this. As experiments get more complex, organizing code into small, well-labelled functions becomes extremely important for maintenance, clarity, and reproducibility.

#### **(F) Instructions + practice block + immediate feedback (good training flow)** ####

In [ ]:
welcome = ''''
Welcome to the Visual Search Task

You will see an assortment of shapes in different positions and orientations.
Most will be 'L' shapes, but some may contain a 'T' shape.

If the T is present, press 'd'.
If the T is absent, press 'a'.

Respond quickly!
Press SPACE to begin 5 practice trials.
'''

instructions = visual.TextStim(win, color='white', text=welcome, units='norm', height=0.05)
instructions.draw()
win.flip()
keys = event.waitKeys(keyList=['space'])
core.wait(0.25)

- **What this code does:**

> This section shows participants clear, step-by-step instructions before the experiment starts.

> The code displays the instructions as a single text block and waits for the participant to press SPACE before continuing.

> The short core.wait(0.25) prevents accidental double keypresses.

- **What I liked/learned:**

> **I like that the flow is really beginner-friendly:** instructions → SPACE → short pause → practice trials.

> This mirrors how real experiments would typically run, where participants get time to read and mentally prepare before the task begins.

> I also like that all the task text is stored in one variable (welcome), which makes it easy to edit later without digging through the main code.

- **What I would keep in mind:**

> There’s a small string-quote typo at the start, but otherwise the structure seems perfect.

> I might eventually move these instructions into a separate function if I wanted to show different messages for practice and real trials, but for a simple experiment, this setup is clean and clear.

#### **(G) End-of-experiment summary (avg RT, accuracy, misses)** ####

In [ ]:
average_rt = round(np.mean(rt_list), 2)
average_corr = round(np.mean(corr_list), 2)
total_miss = sum(miss_rt_list)

avg_rt_text = f'average rt: {average_rt}'
avg_corr_text = f'average correct: {average_corr}'
miss_text = f'no response on {total_miss} trials'
leave_text = 'press SPACE to exit'

cor_avg_back = visual.TextStim(win, text=avg_corr_text, pos=(0, 50), height=20)
rt_avg_back = visual.TextStim(win, text=avg_rt_text, pos=(0, -10), height=20)
miss_tot_back = visual.TextStim(win, text=miss_text, pos=(0, -35), height=20)
exit_text = visual.TextStim(win, text=leave_text, pos=(0, -100), height=20)

cor_avg_back.draw()
rt_avg_back.draw()
miss_tot_back.draw()
exit_text.draw()

win.flip()
keys = event.waitKeys(keyList=['space'])

- **What this code does:**

> This creates a simple summary screen at the end of the experiment showing the participant’s average reaction time, accuracy, and number of missed trials.

>It waits for the participant to press SPACE before closing, giving a clear endpoint to the task.

- **What I liked/learned:**

> I like how this provides an immediate “sanity check” for both the experimenter and the participant.

> I can quickly tell if the task ran properly (for example, if reaction times are reasonable or if there were too many missed trials).

> It also helped me learn how to use lists (rt_list, corr_list, miss_rt_list) to track running results across trials and summarize them cleanly at the end.

- **Why this is a nice touch:**

> Ending with a summary screen feels more polished and professional than just quitting the program.

> It gives closure to the participant (“press SPACE to exit”) and reassures the researcher that data were recorded properly.

> If I expanded this, I might add averages per condition, but as-is, it’s a really clean and informative finish.

#### **(H) Defining stimulus size at the beginning of the experiment** ####

In [ ]:
stim_size = 30
T = visual.ImageStim(win, 'Stimuli/T.png', size=stim_size)
L = visual.ImageStim(win, 'Stimuli/L.png', size=stim_size)

- **What this code does:**

> This defines the size of the target (T) and distractor (L) images at the very start of the script.

> Both images pull from the same stim_size variable, which makes the display consistent and easy to adjust later.

- **What I liked/learned:**

> Instead of manually changing the image size in multiple places, I can adjust all stimuli by changing one value at the top.

> This keeps the code cleaner and reduces errors if I ever need to resize the stimuli for a different display resolution or participant setup.

- **What I would have to change experiment to experiment:**

> If the experiment is moved to a different monitor or scaled for online testing, I only have to tweak stim_size once.

> My code doesn’t use a shared size variable, so resizing would mean updating several separate lines.

> However, if different stim needs different sizes, this method of defining the size once may not be a good idea. 

## **(3) Things I could show my classmate** ##

#### **(A) Explicit quit condition inside the response block** ####

In [ ]:
if "escape" in response:
    win.close()
    core.quit()

- This gives a safe, graceful exit at any point in the task.

- Their version includes an escape condition but buries it inside another function, which makes it less transparent (i.e., this is from my understanding, and I could be wrong).

- It is possible that my approach is easier to spot and modify.

#### **(B) Self-contained logic per trial (i.e., less global dependence)** ####

In [ ]:
def targ_pres(trial_list, total_trials, distract, target, condition_index):
    pres_or_not = random.choice(trial_list)
    trial_list.remove(pres_or_not)
    
    if pres_or_not <= np.median(total_trials):
        targ_there = 0
        stimuli = pos_and_ori(distract, distract, condition_index)
    else:
        targ_there = 1
        stimuli = pos_and_ori(target, distract, condition_index)
    return targ_there, stimuli

- **In my code, most variables (like Set_Size, present, Coordinates_used, and target_position) are defined *inside* the trial loop.**

> This means each trial is self-contained and resets all of its values from scratch. I liked this approach because it made the trial structure easier for me to follow logically — everything needed for that one trial is right there in the loop.

> If something ever goes wrong (like a missed stimulus or crash), it doesn’t affect the next trial since nothing carries over globally.

- **In contrast, this section of my classmate’s code defines the logic for whether a target is present or absent using shared lists (trial_list and total_trials) that get modified each time the function runs.**

> That works just fine, but from my understanding, it would mean that the function depends on those global variables staying in sync.

> For example, removing items from trial_list (trial_list.remove(pres_or_not)) changes its length across trials, so if something breaks mid-run, it could make the block go out of sync or cause inconsistencies, which would make debugging harder later on.

- **As mentioned previously, I really liked that my classmate wrapped this decision logic in a function — that’s very efficient — but I’d personally move the key trial variables into the main trial loop rather than passing them in from outside.**

> Doing so would make it easier to reuse the code in another experiment later, since each trial would generate its own state independently instead of depending on global lists or counters.

#### **(C) Explicit reaction-time clock for the main experimental trials** ####


In [ ]:
rt_clock = core.Clock()

- This makes it obvious that RTs are measured only during the main trials, which, in my opinion, is a good practice.

- However, unless you want to show a participant on the practice trials how fast or slow they were, or something along those lines (i.e., it depends on the task). Possibly to motivate participants to complete the task faster?
  
- My classmate calculated RTs inside the key loop using core.getTime( ) - startTime, which works but is a bit less organized (I could be so wrong about this, but the other way seemed more intuitive).

- Having a dedicated clock object makes timing cleaner and easier to expand later.

## **(4) Questions/thoughts I still have about inputting "Trials per condition"** ##

In [ ]:
dlg.addField('Trials Per Cond:')

- **I understand that this lets the experimenter type how many trials they want for each condition, but I’m still a bit unsure where this number actually gets used in the script.**

> Does it directly control the total number of trials, or does it feed into another loop or function later on?

- **I’m wondering why the experimenter needs to input this value manually rather than having it pre-defined in the code.**

> Is this meant mainly for flexibility during testing, or is there an experimental reason to keep it adjustable?

- **I’m also curious whether participants should ever see or be aware of this number.**

> Since it’s entered in the dialog box before the experiment starts, it seems designed for the experimenter, not the participant — but I’d like to confirm that’s standard practice.

- **Finally, I’d like to understand how “Trials Per Condition” relates to total trial count.**

> For example, if there are multiple set sizes or present/absent conditions, does PsychoPy automatically multiply the entered number across conditions, or does that need to be specified somewhere else?